### Fonctions d'activation
Nous allons commencer par définir les fonctions d'activations ainsi que leurs dérivées

In [ ]:
import numpy as np
import random

def sigmoid(t):
    # Fonction d'activation sigmoïde
    return 1 / (1 + np.exp(-t))

def sigmoid_prime(t):
    # Dérivée de la fonction sigmoïde
    return sigmoid(t) * (1 - sigmoid(t))

def relu(t):
    # Fonction Relu
    return np.maximum(0,t)

def relu_prime(t):
    for i in range(len(t)):
        if t[i]>0 : t[i] = 1
        else: t[i] = 0 # on considère que pour t=0, relu'(t)=0
    return t
    
def identite(t):
    return t

def identite_prime(t):
    return 1

### Classe Réseau de neurones
Nous allons ici créer la classe ReseauNeurones

In [ ]:
"""
    Entrée : la taille du vecteur d'entrée
    Sortie : une instance de réseau de neurones
"""

class ReseauNeurones(object):

    def __init__(self, taille):
        # self.taille est la taille du vecteur d'entrée
        # Dans chaque couche, on génère un nombre aléatoire de neurones
    
        self.a0 = np.array([[]]) # Initialisation de l'activité a0 par un tableau vide
        self.y = 0.0
        # Vecteur de poids et biais de la couche 1
        self.w1 = np.random.randn(random.randint(2, 5), taille)
        self.b1 = np.random.randn(self.w1.shape[0], 1)
        self.z1, self.a1 = np.array([[]]), np.array([[]]) #initialisation de l'activité
        
        # Vecteur de poids et biais de la couche 2
        self.w2 = np.random.randn(random.randint(2, 4), self.w1.shape[0]) # self.w1.shape[0] renvoie le nombre de ligne de w1
        self.b2 = np.random.randn(self.w2.shape[0], 1) # self.w1.shape[0] renvoie le nombre de ligne de w1
        self.z2, self.a2 = np.array([[]]), np.array([[]])
        
        # Vecteur de poids et biais de la couche 3
        self.w3 = np.random.randn(random.randint(2, 3), self.w2.shape[0]) ## self.taille - 3 est un choix arbitraire
        self.b3 = np.random.randn(self.w3.shape[0], 1)
        self.z3, self.a3 = np.array([[]]), np.array([[]])
        
        # Dernière couche
        self.w4 = np.random.randn(1, self.w3.shape[0]) # un seul neurone
        self.b4 = np.random.randn(self.w4.shape[0], 1)
        self.z4, self.a4 = np.array([[]]), np.array([[]])
        
        # Ces varibales nous seront utiles pour la suite
        # On augmente les dimensions des tableaux pour pouvoir utiliser la formule de recurrence de la prop. avant
        self.neurones = [np.array([[]]), self.w1, self.w2, self.w3, self.w4]
        self.biais = [np.array([[]]), self.b1, self.b2, self.b3, self.b4]
        self.preac = [np.array([[]]), self.z1, self.z2, self.z3, self.z4]
        self.activ = [self.a0, self.a1, self.a2, self.a3, self.a4]
        
        self.gw1, self.gw2, self.gw3, self.gw4 = np.array([[]]), np.array([[]]), np.array([[]]), np.array([[]])
        self.gb1, self.gb2, self.gb3, self.gb4 = np.array([[]]), np.array([[]]), np.array([[]]), np.array([[]])
        
        self.grad_w = [np.array([[]]), self.gw1, self.gw2, self.gw3, self.gw4]
        self.grad_b = [np.array([[]]), self.gb1, self.gb2, self.gb3, self.gb4]
        
        #print(self.biais) 

    def propagation_avant(self, X):
        # Propagation avant
        self.activ[0] = X # a0 = X // initialisation
        
        for i in range(1, len(self.neurones)-1):
            self.preac[i] = self.neurones[i].dot(self.activ[i-1])
            self.activ[i] = relu(self.preac[i]) #on a decidé d'utiliser la fonction relu
        
        # Dernière couche
        # l'indice -1 permet de recupérer le dernier élément d'un tableau
        self.preac[-1] = self.neurones[-1].dot(self.activ[-2])
        self.activ[-1] = sigmoid(self.preac[-1]) # on décide de faire de la classification 0/1
        #print(self.preac[-1], '\n\n', self.activ[-1]) # c'était un test
        
        self.y = float(self.activ[-1])  # Prédiction finale
        print('La prédiction est : {}'.format(self.y))
        return self.y

    def retropropagation(self, X, cible, taux=1):
        # Retropropagation du coût
        tmp = self.propagation_avant(X) # On récupère la prédiction de la propagation avant
        
        ga = 2*(tmp - cible) # Initialisation du gradient par rapport à y = a^L
        gz = ga * sigmoid_prime(self.preac[-1])
        self.grad_w[-1] = gz.dot(self.activ[-2].T) * taux
        self.grad_b[-1] = gz * taux
        
        ga = self.neurones[-1].T.dot(gz)
        
        for j in range(len(self.neurones)-2, 0, -1):
        
            gz = ga * relu_prime(self.preac[j])
            self.grad_w[j] = gz.dot(self.activ[j-1].T) * taux
            self.grad_b[j] = gz * taux
            ga = self.neurones[j].T.dot(gz)

    def pas_gradient(self, X, cible, etapes, taux):

        for etape in range(etapes):
            
            # On effectue une rétropropagation et on récupère les dérivées
            self.retropropagation(X, cible, taux)
            
            # Mise à jour des W^i et des b^i
            for k in range(len(self.neurones)-1, 0, -1):
                self.neurones[k] = self.neurones[k] - self.grad_w[k]
                self.biais[k] = self.biais[k] - self.grad_b[k]
            
    def ajouter_couche(self, nb_neurones):
        # On ajoute la nouvelle couche à la fin, avant la dernière couche de prédiction
        
        dim = self.neurones[-2].shape[0] # On récupère le nbre de lignes de l'avant dernière matrice
        self.neurones.insert(-1, np.random.randn(nb_neurones, dim))
        self.biais.insert(-1, np.random.randn(nb_neurones, 1))
        
        # Mise à jour de la dernière couche
        self.neurones[-1] = np.random.randn(1, nb_neurones)
        return "Nouvelle couche ajoutée avec succès !"

## Vous pouvez tester les différentes fonctions

In [ ]:
#here